### Takes a theory and a Z-matrix and returns the polariton surfaces over bond lengths

In [1]:
__authors__ = ["Ruby Manderna, Lane Tolley, Figen Suchanek, Jonathan J. Foley, M. Elijah Wangeman"]
__email__   = ["rmandern@uncc.edu, ptolley1@uncc.edu, suchanekf@wpunj.edu, jfoley19@uncc.edu, mwangema@uncc.edu"]
__credits__ = ["Ruby Manderna, Lane Tolley, Figen Suchanek, Jonathan J. Foley"]
__copyright__ = "(c) 2008-2020, The Psi4Education Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-02-11"

# basic psi4 library
import psi4
# numpy
import numpy as np
# scipy
from scipy.interpolate import InterpolatedUnivariateSpline
# linear algebra package from numpy
from numpy import linalg as LA
# time-dependent scf library from psi4 for computing excited states and transition dipole moments
from psi4.driver.procrouting.response.scf_response import tdscf_excitations
from matplotlib import pyplot as plt
import time

psi4.core.set_output_file('output.dat', False)

In [2]:
def Rabi_Hamiltonian(A_value, omega_value, r_value, g_spline, e_spline, tdm_spline):
    """Function to compute the Rabi Hamiltonian

    Arguments
    ----------
    A_value : float
        fundamental coupling strength
        
    omega_value : float
        photon energy
        
    r_value : float
        value of the bondlength
        
    g_spline : scipy spline object
        spline that is fit to the ground-state potential energy surface
        
    e_spline : scipy spline object
        spline that is fit to the excited-state potential energy surface
        
    tdm_spline : scipy spline object
        spline that is fit to the transition dipole moment surface
        
    Returns
    -------
    H : numpy array
        3x3 Rabi Hamiltonian matrix
    """
    
    # initialize 3x3 Hamiltonian matrix
    H = np.zeros((3,3))
    
    # pre-written first 2 diagonal entries
    H[0,0] = g_spline(r_value)
    H[1,1] = g_spline(r_value) + omega_value
    
    # ==> Code to compute 3rd diagonal entry H[2,2] goes here!
    
    H[2,2] = e_spline(r_value)
    
    # ==> Code to compute off-diagonal entries H[1,2] and H[2,1] goes here!
    
    H[1,2] = A_value * tdm_spline(r_value)
    H[2,1] = A_value * tdm_spline(r_value)
    
    # return the matrix
    return H


In [3]:
def polariton_surfaces(A_value, omega_value, r_values, g_spline, e_spline, tdm_spline):
    """Function to compute the lower- and upper-polariton potential energy surfaces

    Arguments
    ----------
    A_value : float
        fundamental coupling strength
        
    omega_value : float
        photon energy
        
    r_value : float
        value of the bondlength
        
    g_spline : scipy spline object
        spline that is fit to the ground-state potential energy surface
        
    e_splien : scipy spline object
        spline that is fit to the excited-state potential energy surface
        
    tdm_spline : scipy spline object
        spline that is fit to the transition dipole moment surface
        
    Returns
    -------
    E_LP_of_R : numpy array
        lower-polariton potential energy surface defined at each value of r_values
        
    E_UP_of_R : numpy array
        upper-polariton potential energy surface defined at each value of r_values
    """
    # initialize lp and up surfaces
    lp_surface = np.zeros_like(r_values)
    up_surface = np.zeros_like(r_values)
    
    # loop through r values, build Rabi Hamiltonian, diagonalize, and store!
    
    for i in range(0, len(r_values)):
        
        # if there is no coupling, then we don't need to diagonalize anything
        if A_value == 0:
            lp_surface[i] = g_spline(r_values[i]) + omega_value
            up_surface[i] = e_spline(r_values[i])
        
        # otherwise build Hamiltonian and diagonalize
        else:
            # Build the Rabi Hamiltonian
            H = Rabi_Hamiltonian(A_value, omega_value, r_values[i], g_spline, e_spline, tdm_spline)
            
            # diagonalize
            vals, vecs = LA.eigh(H)
            
            # store lp and up values
            lp_surface[i] = vals[1]
            up_surface[i] = vals[2]
    
    # return the surfaces
    return lp_surface, up_surface


In [19]:
zmat1 = """
    Mg
    H 1 """
zmat2 = """
    symmetry c1
    1 1
"""
# theories = ['HF', 'MP2', 'CCSD', 'CCSD(T)']
# basis = ['STO-3G', '3-21G', '6-31G', '6-31G*', '6-31+G*', '6-311G*', 'cc-pVDZ', 'cc-pVTZ', 'cc-pVQZ', 'cc-pV5Z', 'cc-pV6Z']

theories = ['b3lyp', 'mpw1k', 'wb97x-d', 'pbe']
# theories = ['b3lyp']
basis = ['cc-pVDZ', 'cc-PVTZ', 'cc-pVQZ']
# basis = ['cc-pVDZ']

# plot colors for each theory
colors = [
    ['#36013f', '#227b22'],
    ['#bdb5d5', '#00dbff'],
    ['#101820', '#e34234']
    ]

# set the number of electronic states... this is the ground state + n_states more
n_states = 3
# set the number of bond lengths to compute the stretch along
n_geom = 25

rval = 2.5
aval = 0.003

In [20]:
# MAIN LOOP

basis_c = 0
for b in basis:
    for t in theories:
        count = 0
        try:
            # set basis
            psi4.set_options({
                'basis':b
            })

            start = time.process_time()

            # set the number of electronic states... this is the ground state + n_states more
            # we will get 1 excited-state
            n_states = 3

            # set the number of bond lengths to compute the stretch along
            n_geoms = 25

            # initialize geometry list
            geoms = []

            # initialize energy list... note
            # there will be the ground state energy + n_states excited state energies
            Es = np.zeros((n_states+1, n_geoms))

            # initialize z-component of transition dipole list
            mu_z = np.zeros((n_states, n_geoms))

            # generate bond lengths
            rs = []
            for i in range(0,n_geoms):
                rs.append(1.1 + i*0.1)

            # loop over bond lengths
            ctr = 0
            for i in rs:
                # generate the MgH+ molecule using a z-matrix and set the Mg-H+ bond length
                mol = psi4.geometry(zmat1 + str(i) + zmat2)
                # save the geometry
                geoms.append(mol.geometry().to_array())
                psi4.set_options({
                'save_jk': True,
                })  
            
                # calculate and save the ground-state energy and wavefunction
                e, wfn = psi4.energy(t+'/'+b, return_wfn=True, molecule=mol)
                
                # calculate the excited-state energies and save them to a dictionary called 'res'
                res = tdscf_excitations(wfn, states=n_states, triplets = "NONE")
                
                # parse the excitation energies from the 'res' dictionary 
                delta_e = [r["EXCITATION ENERGY"] for r in res]
                
                # parse the transition dipole moment from the 'res' dictionary
                mu = [r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"] for r in res]
                Es[0,ctr] = e
                
                # store the results to the respective arrays
                for j in range(0, n_states):
                    Es[j+1,ctr] = e + delta_e[j]
                    # we only want the z-component which is index 2
                    mu_z[j,ctr] = np.absolute(mu[j][2])

                
                # increment the counter!
                ctr += 1
            
            Eg_array = np.copy(Es[0,:])
            Ee_array = np.copy(Es[1,:])
            tdm_array = np.copy(mu_z[0,:])
            
            Eg_spline = InterpolatedUnivariateSpline(rs, Eg_array, k=3)
            Ee_spline = InterpolatedUnivariateSpline(rs, Ee_array, k=3)
            mu_spline = InterpolatedUnivariateSpline(rs, tdm_array, k=3)

            Eg_val = Eg_spline(rval)
            Ee_val = Ee_spline(rval)
            tdm_val = mu_spline(rval)

            # conversion factor for au -> eV
            au_to_eV = 27.211

            # ==> Code to compute transition energy in atomic units and convert to eV goes here! <== #

            transition_energy_au = Ee_val - Eg_val

            transition_energy_eV = transition_energy_au * au_to_eV

            # Pre-written print the transition energies and check there values!
            # print(transition_energy_au)
            # print(transition_energy_eV)

            H_Rabi = Rabi_Hamiltonian(aval, transition_energy_au, rval, Eg_spline, Ee_spline, mu_spline)

            lp_surface, up_surface = polariton_surfaces(aval, transition_energy_au, rs, Eg_spline, Ee_spline, mu_spline)

            end = time.process_time()

            # time = end - start
            # time = "{time:.3f}"

            # lbl = "Time to Compute: " + time + " seconds."

            plt.plot(rs, Eg_spline(rs)+transition_energy_au, color = colors[basis_c][0], linestyle="--", label="$E_g$ + $\hbar \omega$")
            plt.plot(rs, Ee_spline(rs), color = colors[basis_c][1], linestyle="--", label="$E_e$")
            plt.plot(rs, lp_surface, color=colors[basis_c][0], linestyle="-", label="lower polariton")
            plt.plot(rs, up_surface, color=colors[basis_c][1], linestyle="-", label="upper polariton")

            rabi = "Rabi: " + f"{(up_surface[15] - lp_surface[15]):.5f} $E_h$"
            plt.plot([], [], ' ', label=rabi)

            tm = "TTC: " + f"{end - start:.4f} s"
            plt.plot([], [], ' ', label=tm)

            print(rabi)
            print(start)
            print(end)
            print("Time to Compute: ", end - start, " seconds.")

            plt.grid(True)
            plt.legend()
            plt.xlabel("Bond Length (a.u.)")
            plt.ylabel("Energy ($E_h$)", labelpad=2)
            plt.title("Polariton Surfaces for " + t + "/" + b)
            plt.tight_layout()
            plt.savefig("out/polariton_surfaces_" + t + "_" + b + ".png", transparent=True, dpi=400)
            plt.clf()
            count += 1
        except:
            print("Basis set " + b + " and theory " + t + " failed to converge.")
    basis_c += 1

<>:107: SyntaxWarning: invalid escape sequence '\h'
<>:107: SyntaxWarning: invalid escape sequence '\h'
/var/folders/23/j2wwmprj6vv8knhp4pc510l80000gn/T/ipykernel_20714/1828067565.py:107: SyntaxWarning: invalid escape sequence '\h'
  plt.plot(rs, Eg_spline(rs)+transition_energy_au, color = colors[basis_c][0], linestyle="--", label="$E_g$ + $\hbar \omega$")


Rabi: 0.01412 $E_h$
923.321803
948.67403
Time to Compute:  25.35222699999997  seconds.
Rabi: 0.01431 $E_h$
949.04978
974.630575
Time to Compute:  25.580794999999966  seconds.
Rabi: 0.01423 $E_h$
975.004441
1000.311151
Time to Compute:  25.306709999999953  seconds.
Rabi: 0.01394 $E_h$
1000.677836
1026.31636
Time to Compute:  25.638524000000075  seconds.
Rabi: 0.01417 $E_h$
1026.678089
1069.497319
Time to Compute:  42.81923000000006  seconds.
Rabi: 0.01436 $E_h$
1069.862829
1112.776873
Time to Compute:  42.9140440000001  seconds.
Rabi: 0.01444 $E_h$
1113.142645
1159.731547
Time to Compute:  46.58890200000019  seconds.
Rabi: 0.01398 $E_h$
1160.096683
1205.120588
Time to Compute:  45.02390500000001  seconds.
Rabi: 0.01419 $E_h$
1205.484123
1305.771906
Time to Compute:  100.28778299999999  seconds.
Rabi: 0.01438 $E_h$
1306.13602
1403.870465
Time to Compute:  97.73444500000005  seconds.
Rabi: 0.01447 $E_h$
1404.244444
1505.002632
Time to Compute:  100.75818800000002  seconds.
Rabi: 0.01400 $

<Figure size 640x480 with 0 Axes>